### Install dependencies

In [ ]:
%%bash
git clone https://github.com/KaiyangZhou/Dassl.pytorch.git
cd Dassl.pytorch/
# Install dependencies
pip install -r requirements.txt
# Install this library (no need to re-build if the source code is modified)
python setup.py develop
cd ..
# Install dependencies of MaPLe
pip install -r requirements.txt

### Download dataset (OxfordPets)

In [ ]:
%%bash
cd ../../datasets/data
mkdir oxford_pets
cd oxford_pets
wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
gdown 1501r8Ber4nNKvmlFVQZ8SeUHTcdTTEqs
tar -zxvf images.tar.gz
tar -zxvf annotations.tar.gz

### Run MaPLe

In [ ]:
%%bash
# seed=1
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 1 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 1 ../../datasets/data/

In [ ]:
%%bash
# seed=2
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 2 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 2 ../../datasets/data/

In [ ]:
%%bash
# seed=3
# trains and evaluates on base classes
bash scripts/maple/base2new_train_maple.sh oxford_pets 3 ../../datasets/data/
# evaluates on novel classes
bash scripts/maple/base2new_test_maple.sh oxford_pets 3 ../../datasets/data/

## Prompt Learning

In [1]:
%%bash
bash scripts/maple_prompt_scene/base2new_train_maple_prompt_scene.sh replica 1 ../../datasets/data/

Run this job and save the output to output/base2new/train_base/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1
Error while terminating subprocess (pid=2879446): 


In [3]:
%%bash
bash scripts/maple_prompt_scene/base2new_test_maple_prompt_scene.sh replica 1 ../../datasets/data/

Evaluating model
Runing the first phase job and save the output to output/base2new/test_new/replica/shots_16/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Error while terminating subprocess (pid=2879966): 


In [10]:
%%bash
bash scripts/maple_prompt_scene/xd_train_maple_prompt_scene.sh replica 1 ../../datasets/data/

Run this job and save the output to output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: False
head: 
load_epoch: None
model_dir: 
no_train: False
opts: ['DATASET.NUM_SHOTS', '16']
output_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed1
resume: 
root: ../../datasets/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIFAR_C_LEVEL: 1
  CIFAR_C_TYPE: 
  NAME: Repl

+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
Building transform_test
+ resize the smaller edge to 224
+ 224x224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
---------  -------
Dataset    Replica
# classes  5
# train_x  11
# val      2
# test     2
---------  -------
Loading CLIP (backbone: ViT-B/16)
Building custom CLIP
MaPLe design: Multi-modal Prompt Learning
Initial context: "a photo of a"
Number of MaPLe context words (tokens): 2
Turning off gradients in both the image and the text encoder
Parameters to be updated: {'prompt_learner.proj.weight', 'prompt_learner.compound_prompts_text.4', 'prompt_learner.compound_prompt_projections.3.weight', 'prompt_learner.compound_prompt_projections.0.bias', 'prompt_learner.ctx', 'prompt_learner.compound_prompt_projections.5.weight', 'prompt_learner.compound_

100%|██████████| 1/1 [00:00<00:00,  3.69it/s]


output: tensor([[4.3750, 5.0859, 3.8438, 3.8203, 5.3633],
        [4.5977, 5.6211, 5.2305, 7.1211, 6.8438]], device='cuda:0',
       dtype=torch.float16)
label: tensor([1, 4], device='cuda:0')
=> result
* total: 2
* correct: 0
* accuracy: 0.0%
* error: 100.0%
* macro_f1: 0.0%
Elapsed: 0:00:02


In [11]:
%%bash
bash scripts/maple_prompt_scene/xd_test_maple_prompt_scene.sh replica 1 ../../datasets/data/

Run this job and save the output to output/evaluation/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/replica/seed1


/cluster/54/nanriayaka/ml3d/models/multimodal-prompt-learning/clip/clip.py:23: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


Setting fixed seed: 1
***************
** Arguments **
***************
backbone: 
config_file: configs/trainers/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets.yaml
dataset_config_file: configs/datasets/replica.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: output/replica/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed1
no_train: False
opts: []
output_dir: output/evaluation/MaPLePromptScene/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/replica/seed1
resume: 
root: ../../datasets/data/
seed: 1
source_domains: None
target_domains: None
trainer: MaPLePromptScene
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 4
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL

Building transform_test
+ resize the smaller edge to 224
+ 224x224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
---------  -------
Dataset    Replica
# classes  5
# train_x  11
# val      2
# test     2
---------  -------
Loading CLIP (backbone: ViT-B/16)
Building custom CLIP
MaPLe design: Multi-modal Prompt Learning
Initial context: "a photo of a"
Number of MaPLe context words (tokens): 2
Turning off gradients in both the image and the text encoder
Parameters to be updated: {'prompt_learner.proj.weight', 'prompt_learner.compound_prompts_text.2', 'prompt_learner.compound_prompts_text.5', 'prompt_learner.compound_prompt_projections.1.bias', 'prompt_learner.compound_prompt_projections.2.weight', 'prompt_learner.compound_prompt_projections.0.bias', 'prompt_learner.compound_prompts_text.4', 'prompt_learner.compound_prompts_text.1', 'prompt_learner.compound_prompts_text.6', 'prompt_learner.

100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


output: tensor([[4.3750, 5.0859, 3.8438, 3.8203, 5.3633],
        [4.5977, 5.6211, 5.2305, 7.1211, 6.8438]], device='cuda:0',
       dtype=torch.float16)
label: tensor([1, 4], device='cuda:0')
=> result
* total: 2
* correct: 0
* accuracy: 0.0%
* error: 100.0%
* macro_f1: 0.0%
